In [69]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
import math
import importlib
import Spatial_attention_model
importlib.reload(Spatial_attention_model)
from Spatial_attention_model import ShallowAttentionNet
from shallow_laurits import ShallowFBCSPNet
from torchinfo import summary
from weight_init import init_weights

In [70]:
in_chans = 22
n_classes = 4
input_window_samples = 1125
model = ShallowAttentionNet(in_chans,n_classes,input_window_samples)
model.apply(init_weights)

ShallowAttentionNet(
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (spatial_att): SpatialAttention(
    (conv): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (pool): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (batch_norm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 20), stride=(1, 20), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=5940, out_features=4, bias=True)
)

In [71]:
print(model)

ShallowAttentionNet(
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (spatial_att): SpatialAttention(
    (conv): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (pool): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (batch_norm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 20), stride=(1, 20), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=5940, out_features=4, bias=True)
)


In [72]:
summary(model,input_size=(2, 22, 1125),col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowAttentionNet                      [2, 22, 1125]             [2, 4]                    --                        --
├─Conv2d: 1-1                            [2, 1, 22, 1125]          [2, 10, 22, 1101]         260                       [1, 25]
├─SpatialAttention: 1-2                  [2, 10, 22, 1101]         [2, 10, 22, 550]          --                        --
│    └─Conv2d: 2-1                       [2, 10, 22, 1101]         [2, 10, 22, 1101]         100                       [1, 1]
│    └─MaxPool2d: 2-2                    [2, 10, 22, 1101]         [2, 10, 22, 550]          --                        [1, 2]
├─BatchNorm2d: 1-3                       [2, 10, 22, 550]          [2, 10, 22, 550]          20                        --
├─AvgPool2d: 1-4                         [2, 10, 22, 550]          [2, 10, 22, 27]           --                        [1, 